In [13]:
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.model_selection import cross_val_predict, StratifiedGroupKFold, GroupKFold
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import ConfusionMatrixDisplay

from pptoolbox.platform.preprocessing import preprocess_v2
from pptoolbox.platform.automl_v4 import AutoML_v4
from pptoolbox.platform.automl_v4.pipelines import META_REGRESSION_PIPELINES

import os
# import sys
# sys.path.append("../src")
# from utils.pipelines_meta import META_REGRESSION_PIPELINES # Shifted into pptoolbox 1.3.1


### Helper Utils

In [14]:
# Dumping meta utils here. 
# Algorithm design still undergoing testing. not yet stable for pptoolbox?
import warnings
from typing import Literal, Optional
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import BaseCrossValidator, cross_val_predict


AUTOML_MODE_OPTIONS = ["classify", "profile"]

def create_meta_datasets_splits(
    estimators: list[list[Pipeline]],
    datasets: list[pd.DataFrame],
    y: pd.Series,
    groups: np.ndarray,
    outer_cv: BaseCrossValidator,
    inner_cv: BaseCrossValidator,
    mode: Literal[AUTOML_MODE_OPTIONS] = "classify",
) -> tuple[pd.DataFrame, pd.Series, np.ndarray]:
    """
    Custom pre-processing for meta-model cross validation flow
    This flow assumes we maintain the same split indexes across all hyperparams to be tested.
    Because of this, we can pre-calculate the features being input to meta-model
    Instead of re-fitting the same estimator at each fold
    """
    if mode not in AUTOML_MODE_OPTIONS:
        raise ValueError(f"Unexpected mode: {mode}. Expected {AUTOML_MODE_OPTIONS}")
    if len(datasets) == 1:
        raise ValueError(
            "This is meant for data fusion. Check your input is a List of pandas.DataFrame"
        )

    for X in datasets[1:]:
        if len(X) != len(datasets[0]):
            raise IndexError(
                f"Lengths of input datasets do not match ({datasets[0].shape[0]}), ({X.shape[0]})"
            )

    for i, estimator_list in enumerate(estimators):
        if type(estimator_list) != list:
            if type(estimator_list) == Pipeline:
                warnings.warn("Single estimator detected. Converting to list")
                estimators[i] = [estimator_list]
            else:
                raise ValueError("estimators should be a list of lists of Pipelines")

    if len(datasets) != len(estimators):
        raise ValueError(
            "Length of datasets and estimators do not match. Check your inputs"
        )

    if mode == "classify":
        n_classes = len(np.unique(y))
    else:
        n_classes = 1

    n = len(datasets[0])
    m_list = [0] + [len(estimator_list) for estimator_list in estimators]
    m = sum(m_list) * n_classes
    k = outer_cv.get_n_splits()

    new_indices = np.concatenate([datasets[0].index] * k)
    meta_X = np.zeros((n * k, m))
    meta_y = pd.Series(np.concatenate([y] * k), index=new_indices)
    splits_idx = np.zeros((n * k, 2))

    for fold_idx, (train_idx, test_idx) in enumerate(
        outer_cv.split(datasets[0], y, groups=groups)
    ):
        y_train = y.iloc[train_idx]
        splits_idx[n * fold_idx : n * (fold_idx + 1), 0] = fold_idx
        splits_idx[test_idx + (n * fold_idx), 1] = 1

        for i in range(len(datasets)):
            X = datasets[i]
            for j, estimator in enumerate(estimators[i]):
                X_train = X.iloc[train_idx]
                X_test = X.iloc[test_idx]

                if mode == "classify":
                    # Classification Flow
                    preds_train = cross_val_predict(
                        estimator,
                        X_train,
                        y_train,
                        groups=X_train.index,
                        cv=inner_cv,
                        method="predict_proba",
                    )
                    estimator.fit(X_train, y_train)
                    preds_test = estimator.predict_proba(X_test)
                else:
                    # Regression Flow
                    preds_train = cross_val_predict(
                        estimator,
                        X_train,
                        y_train,
                        groups=X_train.index,
                        cv=inner_cv,
                        method="predict",
                    )
                    estimator.fit(X_train, y_train)
                    preds_test = estimator.predict(X_test)

                meta_X[
                    train_idx + n * fold_idx,
                    n_classes * (j + m_list[i]) : n_classes * (j + 1 + m_list[i]),
                ] = preds_train.reshape(X_train.shape[0], n_classes)
                meta_X[
                    test_idx + n * fold_idx,
                    n_classes * (j + m_list[i]) : n_classes * (j + 1 + m_list[i]),
                ] = preds_test.reshape(X_test.shape[0], n_classes)

    meta_X = pd.DataFrame(meta_X, index=new_indices)

    return meta_X, meta_y, splits_idx


class CustomMetaSplitter(BaseCrossValidator):
    def __init__(self, split_idx: np.ndarray):
        self.splits = split_idx
        self.n_splits = len(np.unique(split_idx[:, 0]))

    def get_n_splits(
        self, X: Optional = None, y: Optional = None, groups: Optional = None
    ) -> int:
        return self.n_splits

    def split(self, X: Optional = None, y: Optional = None, groups: Optional = None):
        for i in range(self.n_splits):
            train_idx = np.where((self.splits[:, 0] == i) & (self.splits[:, 1] == 0))[0]
            test_idx = np.where((self.splits[:, 0] == i) & (self.splits[:, 1] == 1))[0]
            yield train_idx, test_idx



In [15]:
def initialise_trainer(task_type ='classify', log=False, experiment_name='', mlflow_uri=None, *args, **kwargs):
    trainer = AutoML_v4(task_type = task_type, log = log, *args, **kwargs, exploration_runs=200)
    if log == True:
        trainer.set_log_config(
            experiment_name = f"{experiment_name}",
            mlflow_uri = mlflow_uri
        )
    return trainer

In [16]:
def generate_metafeatures_profile(datasets, estimators, y, cv=GroupKFold(n_splits=3)):
    n_classes = 1
    for i, estimator_list in enumerate(estimators):
        if type(estimator_list) != list:
            if type(estimator_list) == Pipeline:
                # warnings.warn("Single estimator detected. Converting to list")
                estimators[i] = [estimator_list]
            else:
                raise ValueError("estimators should be a list of lists of Pipelines")
    n = len(datasets[0])
    m_list = [0] + [len(estimator_list) for estimator_list in estimators]
    m = sum(m_list) * n_classes

    X_meta = np.zeros([n,m])
    for i in range(len(datasets)):
        X = datasets[i]
        for j, estimator in enumerate(estimators[i]):
            y_preds = cross_val_predict(estimator, X, y, groups=X.index, cv=cv, method='predict')

            X_meta[
                :,
                n_classes * (j + m_list[i]) : n_classes * (j + 1 + m_list[i]),
            ] = y_preds.reshape(n, n_classes)
            pass

    return X_meta


In [17]:
"""Combined file simulating platform utils file."""
import pandas as pd
import numpy as np

import os
import pickle as pkl
from typing import Annotated, List, Tuple, Optional

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import confusion_matrix
from pptoolbox.platform.automl_v4 import cross_group_predict
from pptoolbox.platform.automl_v4 import AutoML_v4

# Dump to file
def model_dump_regression(trainer: AutoML_v4, estimator, y: pd.Series, output_dir: str, suffix: str=""):
    # Dumps with this function will be done on a single label basis
    # This is only a partial dump. Final model needs to be selected by higher level function after reviewing metrics
    label = y.name

    pkl.dump(
        estimator, 
        open(os.path.join(output_dir, f"tasteprofile_model_{label}{suffix}.pkl"), "wb")
    )
    pkl.dump(
        trainer, 
        open(os.path.join(output_dir, f"trainer_{label}{suffix}.pkl"), "wb")
    )

    # Get CV metrics from AutoML trainer
    metrics = generate_regression_metrics(trainer, y)

    return metrics

# Get Regression CV metrics from AutoML trainer
def generate_regression_metrics(trainer, y):
    best_trial = trainer.get_best_trial()
    best_cv_results = best_trial["result"]["cv_results"]

    pmae_range = abs(y.max() - y.min())
    cv_mae = -best_cv_results["test_neg_mean_absolute_error"]
    cv_pmae = (cv_mae * 100) / pmae_range

    metrics = {
        "model": trainer.get_pipeline_name(), 
        "param": y.name,
        "r2": best_cv_results["test_r2"],
        "mae": -best_cv_results["test_neg_mean_absolute_error"],
        "mse": -best_cv_results["test_neg_mean_squared_error"],
        "rmse": -best_cv_results["test_neg_root_mean_squared_error"],
        "max_error": best_cv_results["test_max_error"],
        "pmae": cv_pmae,
    }

    return metrics

from pptoolbox.visualization.reg_visualizers import get_reg_figures
def dump_images_regression(estimator, X:pd.DataFrame, y:pd.Series, output_dir:str, suffix:str='None'):

    fig1, fig2 = get_reg_figures(
        model=estimator,
        Xtrain=X,
        ytrain=y,
        test=False,
    )
    im_dir = Path(output_dir / 'images')
    im_dir.mkdir(exist_ok=True)
    label = y.name
    
    # Prediction error fig (Actual vs Predicted)
    fig1.update_layout(title=suffix)
    output_path = Path(im_dir / f"prederror_{label}{suffix}.html")
    fig1.write_html(output_path)

    # Residuals fig
    fig2.update_layout(title=suffix)
    output_path = Path(im_dir / f"residuals_{label}{suffix}.html")
    fig2.write_html(output_path)

    return None


### Init

In [18]:
# Initialise configs
sql_response_file = Path("../data/processed/input.csv")
labels_file = Path("../data/processed/label.csv")
# output_dir = Path("../model/predictions_profile")
experiment_name = "ketchup profile"

# Rename the qmini, qneo columns to align with automl_orca hardcoded names
# mapper = {
#     "calc_data_mini": "raw_data_mini",
#     "calc_data_neo": "raw_data_neo",
# }
# input_df = input_df.rename(mapper=mapper, axis=1)

input_df = pd.read_csv(sql_response_file) 
labels_df = pd.read_csv(labels_file)

labels_df = labels_df[['lot_id', 'price per weight',  'Days to Expiry', 'avg mid','salt (mg)','carbohydrates (g)','protein (g)']]

X_visnir, X_exnir, y, datatype = preprocess_v2(input_df, labels_df)

N_SPLITS = 3
DEFAULT_CV = GroupKFold(n_splits=N_SPLITS)

### Training single

In [ ]:
label = list(y.columns)[0]
input_y = y[label] # Show example for single column only

In [ ]:
# Run visnir segment
trainer = initialise_trainer(task_type='profile', log=False)
trainer.fit(X_visnir, input_y, kfold=DEFAULT_CV)
score_visnir = trainer.get_best_performance()
best_visnir = trainer.get_pipeline()
metrics_visnir = generate_regression_metrics(trainer, input_y)
trainer_visnir = trainer
base_visnir = [best_visnir]

In [ ]:
# Run exnir segment
trainer = initialise_trainer(task_type='profile', log=False)
trainer.fit(X_exnir, input_y, kfold=DEFAULT_CV)
score_exnir = trainer.get_best_performance()
best_exnir = trainer.get_pipeline()
metrics_exnir = generate_regression_metrics(trainer, input_y)
trainer_exnir = trainer
base_exnir = [best_exnir]

In [ ]:
# Prepare dataset for metamodel segment
datasets = [X_visnir, X_exnir]
estimators = [base_visnir, base_exnir] 
X_meta, y_meta, split_idx = create_meta_datasets_splits(
    estimators=estimators, 
    datasets=datasets, 
    y=input_y, 
    groups=X_visnir.index, 
    outer_cv=DEFAULT_CV, 
    inner_cv=DEFAULT_CV,
    mode='profile',
)

# Run metamodel segment
cv_meta = CustomMetaSplitter(split_idx = split_idx)
trainer = initialise_trainer(task_type='profile', log=False)
trainer.fit(X_meta, y_meta, kfold=cv_meta, search_space=META_REGRESSION_PIPELINES, groups=X_meta.index)
score_meta = trainer.get_best_performance()
metrics_meta = generate_regression_metrics(trainer, input_y)

X_meta_final = generate_metafeatures_profile(datasets, estimators, y=input_y)
best_meta = trainer.get_pipeline()
best_meta.fit(X_meta_final, input_y)
trainer_meta = trainer
print("Done")

In [ ]:
# Review metrics without dumping
import pprint
print(score_visnir, score_exnir, score_meta)

print()
print('visnir')
pprint.pprint(metrics_visnir)

print()
print('exnir')
pprint.pprint(metrics_exnir)

print()
print('metamodel')
pprint.pprint(metrics_meta)



In [ ]:
fig1, fig2 = get_reg_figures(
    model=best_visnir,
    Xtrain=X_visnir,
    ytrain=input_y,
    test=False,
)

# Prediction error fig (Actual vs Predicted)
fig1.update_layout(title='visnir')
fig1.show()

# Residuals fig
fig2.update_layout(title='visnir')
fig2.show()

In [ ]:
fig1, fig2 = get_reg_figures(
    model=best_exnir,
    Xtrain=X_exnir,
    ytrain=input_y,
    test=False,
)

# Prediction error fig (Actual vs Predicted)
fig1.update_layout(title='exnir')
fig1.show()

# Residuals fig
fig2.update_layout(title='exnir')
fig2.show()

In [ ]:
fig1, fig2 = get_reg_figures(
    model=best_meta,
    Xtrain=X_meta_final,
    ytrain=input_y,
    test=False,
)

# Prediction error fig (Actual vs Predicted)
fig1.update_layout(title='meta')
fig1.show()

# Residuals fig
fig2.update_layout(title='meta')
fig2.show()


In [ ]:
# Export Single
# Dump metrics to folder
output_dir = Path("../tests/testdata/predictions_profile/")
output_dir.mkdir(exist_ok=True, parents=True)

_ = model_dump_regression(
    trainer=trainer_visnir, 
    estimator=best_visnir, 
    y=input_y, output_dir=output_dir, suffix="_visnir",
)

_ = model_dump_regression(
    trainer=trainer_exnir, 
    estimator=best_exnir, 
    y=input_y, output_dir=output_dir, suffix="_exnir",
)

_ = model_dump_regression(
    trainer=trainer_meta, 
    estimator=best_meta, 
    y=input_y, output_dir=output_dir, suffix="_meta",
)

# Full Loop with dumps


In [19]:
from pptoolbox.platform.utils import export_pi_df, initialize_pi_df

# Export EVERYTHING, images included as plotly html
# Dump metrics to folder
output_dir = Path("../model/predictions_profile/")
output_dir.mkdir(exist_ok=True, parents=True)

In [20]:
y.columns

Index(['price per weight', 'Days to Expiry', 'avg mid', 'salt (mg)',
       'carbohydrates (g)', 'protein (g)'],
      dtype='object')

In [21]:
# this flow considers each column in y_DataFrame as regression labels
# Will loop through each column and generate outputs for each

# Will also generate a consolidated metrics file for each data type (visnir, exnir, meta)

metrics_visnir = []
metrics_exnir = []
metrics_meta = []

metrics_best = []
labels = list(y.columns)
for label in labels:
    input_y = y[label]

    print (label)

    # Run visnir segment
    trainer = initialise_trainer(task_type='profile', log=False)
    trainer.fit(X_visnir, input_y, kfold=DEFAULT_CV)
    score_visnir = trainer.get_best_performance()
    best_visnir = trainer.get_pipeline()
    base_visnir = [best_visnir]

    metric_visnir = model_dump_regression(
        trainer=trainer, 
        estimator=trainer.get_pipeline(), 
        y=input_y, output_dir=output_dir, suffix="_visnir",
    )
    metrics_visnir.append(metric_visnir)

    dump_images_regression(
        estimator=trainer.get_pipeline(), 
        X=X_visnir, 
        y=input_y, output_dir=output_dir, suffix="_visnir",
    )

    # Run exnir segment
    trainer = initialise_trainer(task_type='profile', log=False)
    trainer.fit(X_exnir, input_y, kfold=DEFAULT_CV)
    score_exnir = trainer.get_best_performance()
    best_exnir = trainer.get_pipeline()
    base_exnir = [best_exnir]

    metric_exnir = model_dump_regression(
        trainer=trainer, 
        estimator=trainer.get_pipeline(), 
        y=input_y, output_dir=output_dir, suffix="_exnir",
    )
    metrics_exnir.append(metric_exnir)

    dump_images_regression(
        estimator=trainer.get_pipeline(), 
        X=X_exnir, 
        y=input_y, output_dir=output_dir, suffix="_exnir",
    )

    # Prepare dataset for metamodel segment
    datasets = [X_visnir, X_exnir]
    estimators = [base_visnir, base_exnir] 
    X_meta, y_meta, split_idx = create_meta_datasets_splits(
        estimators=estimators, 
        datasets=datasets, 
        y=input_y, 
        groups=X_visnir.index, 
        outer_cv=DEFAULT_CV, 
        inner_cv=DEFAULT_CV,
        mode='profile',
    )

    # Run metamodel segment
    cv_meta = CustomMetaSplitter(split_idx = split_idx)
    trainer = initialise_trainer(task_type='profile', log=False)
    trainer.fit(X_meta, y_meta, kfold=cv_meta, search_space=META_REGRESSION_PIPELINES, groups=X_meta.index)
    score_meta = trainer.get_best_performance()

    X_meta_final = generate_metafeatures_profile(datasets, estimators, y=input_y)
    best_meta = trainer.get_pipeline()
    best_meta.fit(X_meta_final, input_y)

    metric_meta = model_dump_regression(
        trainer=trainer, 
        estimator=trainer.get_pipeline(), 
        y=input_y, output_dir=output_dir, suffix="_meta",
    )
    metrics_meta.append(metric_meta)

    dump_images_regression(
        estimator=trainer.get_pipeline(), 
        X=X_meta_final, 
        y=input_y, output_dir=output_dir, suffix="_meta",
    )

    best_model = np.argmax(np.array([score_visnir, score_exnir, score_meta]))
    # argmax, Most positive neg-MSE wins
    metric_options = {
        0: metric_visnir,
        1: metric_exnir,
        2: metric_meta,
    }
    metrics_best.append(metric_options[best_model])
    continue

# Export pi_df and common metrics for best models
pi_df = initialize_pi_df(labels)
export_pi_df(pi_df, output_dir)
metrics_df = pd.DataFrame(metrics_best)
metrics_df.to_csv(os.path.join(output_dir, "metrics.csv"), index=False) 

metrics_df = pd.DataFrame(metrics_visnir)
metrics_df.to_csv(os.path.join(output_dir, "metrics_visnir.csv"), index=False)
metrics_df = pd.DataFrame(metrics_exnir)
metrics_df.to_csv(os.path.join(output_dir, "metrics_exnir.csv"), index=False)
metrics_df = pd.DataFrame(metrics_meta)
metrics_df.to_csv(os.path.join(output_dir, "metrics_meta.csv"), index=False)


price per weight
100%|██████████| 350/350 [02:17<00:00,  2.54trial/s, best loss: 126.76056728944715]
Days to Expiry
 82%|████████▏ | 287/350 [01:58<00:25,  2.43trial/s, best loss: 30501.396802709394]
avg mid
100%|██████████| 350/350 [02:32<00:00,  2.29trial/s, best loss: 14.513502994076136]
salt (mg)
 88%|████████▊ | 307/350 [02:20<00:19,  2.18trial/s, best loss: 683082.5825558343]
carbohydrates (g)
 83%|████████▎ | 289/350 [02:29<00:31,  1.93trial/s, best loss: 61.6378232524456]
protein (g)
100%|██████████| 350/350 [02:57<00:00,  1.97trial/s, best loss: 0.6156630304981954]
